In [1]:
!pip install transformers torch datasets scikit-learn pandas accelerate peft regex

In [2]:
import pandas as pd
import torch
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from datasets import Dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import time
from google.colab import files

## Загрузка данных

In [4]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
categories = ['бытовая техника', 'обувь', 'одежда', 'посуда', 'текстиль', 'товары для детей', 'украшения и аксессуары', 'электроника', 'нет товара']

## Промпт инжиниринг

In [5]:
few_shot_examples = [
    ("Заказали 14.10.2017, получили 25.10.2017. На мой размер 42, широкий как мешок. По поводу качества хороший пуховик. Мех натуральный, съемный.", "одежда", "Шаг1: Ключевые: пуховик, мех. Шаг2: Верхняя одежда. Шаг3: Товар упомянут. Шаг4: одежда."),
    ("Товар не получила, деньги не вернули! Открыла спор, но его закрыли.", "нет товара", "Шаг1: Нет ключевых слов о товаре. Шаг2: N/A. Шаг3: Только спор/доставка. Шаг4: нет товара."),
    ("На 42-44 маленькая ОЧЕНЬ! Рукава короткие! ... подождите, это обувь? Нет, ботинки пришли с дыркой, низ рваный.", "обувь", "Шаг1: Ключевые: ботинки, низ рваный. Шаг2: Обувь. Шаг3: Товар упомянут. Шаг4: обувь."),
    ("Посуда пришла с браком, тарелки треснули, кастрюля тонкая.", "посуда", "Шаг1: Ключевые: посуда, тарелки, кастрюля. Шаг2: Кухонная утварь. Шаг3: Товар упомянут. Шаг4: посуда."),
    ("Ткань прозрачная, постельное белье видно нижнее, но мягкое.", "текстиль", "Шаг1: Ключевые: постельное белье, ткань. Шаг2: Текстиль, не одежда. Шаг3: Товар упомянут. Шаг4: текстиль."),
    ("Игрушка для ребенка сломалась, одежда для малышей мала, но качество хорошее.", "товары для детей", "Шаг1: Ключевые: игрушка, одежда для малышей. Шаг2: Детские товары. Шаг3: Товар упомянут. Шаг4: товары для детей."),
    ("Серьги красивые, но аксессуары с браком, сумка не застегивается.", "украшения и аксессуары", "Шаг1: Ключевые: серьги, аксессуары, сумка. Шаг2: Ювелирка/аксессуары. Шаг3: Товар упомянут. Шаг4: украшения и аксессуары."),
    ("Чайник сломался через неделю, бытовая техника низкого качества.", "бытовая техника", "Шаг1: Ключевые: чайник, бытовая техника. Шаг2: Приборы. Шаг3: Товар упомянут. Шаг4: бытовая техника.")
]

In [6]:
prompt_template = """<role>Ты эксперт по анализу отзывов с маркетплейсов (AliExpress, Wildberries). Твоя специализация — классификация по категориям товаров на основе ключевых упоминаний. Будь точным: анализируй только текст отзыва, игнорируй эмоции/доставку, если нет товара.</role>

<categories>
Доступные категории (выбери ровно одну, основываясь на товаре):
{categories_list}
</categories>

<instructions>
Выполни классификацию в 4 шага (мысленно, не пиши их в ответе, кроме как в примерах):
1. Извлеки ключевые слова о товаре (e.g., "пуховик" — одежда; "чайник" — бытовая техника).
2. Сопоставь с категорией: используй описания выше. Если товар граничит (e.g., детская одежда — "товары для детей", если указано; иначе "одежда").
3. Проверь на "нет товара": если отзыв только о доставке/качестве без конкретики — это класс.
4. Выбери одну категорию. Если неоднозначно — fallback на "одежда" (самая частая в отзывах).
</instructions>

<examples>
{examples_str}
</examples>

<query>
Новый отзыв: {text}
</query>

<output_format>
Ответь только в формате: <category>[название категории]</category>
Не добавляй объяснения или текст.
</output_format>"""

In [7]:
def build_prompt(text, examples):
    categories_list = "\n".join([f"- {cat}: {desc}" for cat, desc in [
        ('бытовая техника', 'бытовые приборы (чайник, пылесос, стиральная машина)'),
        ('обувь', 'обувь (ботинки, кроссовки, туфли)'),
        ('одежда', 'одежда (футболки, штаны, платья, куртки, шапки, белье)'),
        ('посуда', 'кухонная посуда (тарелки, кастрюли, стаканы)'),
        ('текстиль', 'текстильные изделия (постельное белье, полотенца, шарфы, не одежда)'),
        ('товары для детей', 'детские товары (игрушки, одежда для малышей, коляски)'),
        ('украшения и аксессуары', 'ювелирка и аксессуары (серьги, сумки, часы, не одежда)'),
        ('электроника', 'гаджеты (телефон, наушники, зарядки)'),
        ('нет товара', 'если нет упоминания товара (только доставка, спор, возврат денег)')
    ]])
    examples_str = "\n".join([
        f"<example>\n<отзыв>{ex[0]}</отзыв>\n<cot>{ex[2]}</cot>\n<category>{ex[1]}</category>\n</example>"
        for ex in examples
    ])
    return prompt_template.format(categories_list=categories_list, examples_str=examples_str, text=text)

## Модель

In [ ]:
model_name = "sberbank-ai/ruT5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1, max_new_tokens=20, do_sample=False)

In [9]:
def classify_review(text, examples):
    prompt = build_prompt(text, examples)
    result = generator(prompt)[0]['generated_text']

    match = re.search(r'<category>([^<]+)</category>', result)
    category = match.group(1).strip() if match else "нет товара"
    if category not in categories:
        category = "нет товара"
    return category

In [ ]:
predictions = []
start_time = time.time()
for idx, row in test_df.iterrows():
    pred = classify_review(row['text'], few_shot_examples)
    predictions.append(pred)
    if (idx + 1) % 100 == 0:
        elapsed = (time.time() - start_time) / (idx + 1)
        print(f"Processed {idx+1}/{len(test_df)}, avg time: {elapsed:.2f}s")
avg_time = (time.time() - start_time) / len(test_df)
print(f"Avg time per example: {avg_time:.2f}s (must <5s)")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processed 100/7276, avg time: 1.88s
Processed 200/7276, avg time: 1.76s
Processed 300/7276, avg time: 1.72s
Processed 400/7276, avg time: 1.70s
Processed 500/7276, avg time: 1.68s
Processed 600/7276, avg time: 1.68s
Processed 700/7276, avg time: 1.67s
Processed 800/7276, avg time: 1.66s
Processed 900/7276, avg time: 1.66s
Processed 1000/7276, avg time: 1.66s
Processed 1100/7276, avg time: 1.65s
Processed 1200/7276, avg time: 1.65s
Processed 1300/7276, avg time: 1.65s
Processed 1400/7276, avg time: 1.65s
Processed 1500/7276, avg time: 1.65s
Processed 1600/7276, avg time: 1.65s
Processed 1700/7276, avg time: 1.64s
Processed 1800/7276, avg time: 1.64s
Processed 1900/7276, avg time: 1.64s
Processed 2000/7276, avg time: 1.64s
Processed 2100/7276, avg time: 1.64s
Processed 2200/7276, avg time: 1.64s
Processed 2300/7276, avg time: 1.64s
Processed 2400/7276, avg time: 1.64s
Processed 2500/7276, avg time: 1.64s
Processed 2600/7276, avg time: 1.64s
Processed 2700/7276, avg time: 1.64s
Processed 

In [ ]:
submission = pd.DataFrame({'category': predictions})
submission.to_csv('data/submission.csv', index=False)
files.download('data/submission.csv')
print("Submission saved and downloaded!")

In [ ]:
train_sample = train_df.head(200).copy()
keyword_map = {
    'одежда': ['футболка', 'пуховик', 'штаны', 'платье', 'блузка', 'юбка', 'кофта', 'свитер'],
    'нет товара': ['доставка', 'спор', 'деньги не вернули', 'не пришла'],
    'обувь': ['обувь', 'ботинки', 'кроссовки'],
}

def dummy_label(text):
    text_lower = text.lower()
    for cat, kws in keyword_map.items():
        if any(kw in text_lower for kw in kws):
            return cat
    return 'одежда'

true_labels = [dummy_label(t) for t in train_sample['text']]
pred_labels = [classify_review(t, few_shot_examples) for t in train_sample['text']]
f1 = f1_score(true_labels, pred_labels, average='weighted')
print(f"Simulated Weighted F1 on train sample: {f1:.3f}")